In [1]:
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/bjn_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/eng_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/ind_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/jav_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/zsm_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/zho_Hans.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/tam_Taml.dev

In [2]:
with open('eng_Latn.dev') as fopen:
    en = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
en_, ms_ = [], []
for i in range(len(en)):
    if len(en[i]) and len(ms[i]):
        en_.append(en[i])
        ms_.append(ms[i])

In [3]:
len(en_), len(ms_)

(997, 997)

In [4]:
from tqdm import tqdm
import requests
import os
import json

In [5]:
!mkdir small-en-ms

In [6]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-small-malaysian-translation-v2')
model = T5ForConditionalGeneration.from_pretrained('mesolitica/nanot5-small-malaysian-translation-v2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
ms_[0]

'Pada hari Isnin, Saintis daripada Sekolah Perubatan Universiti Stamford mengumumkan penemuan alat diagnostik baru yang boleh mengasingkan sel-sel mengikut jenis: cip kecil yang boleh dicetak yang boleh dihasilakn menggunakan pencetak standard inkjet untuk kira-kira satu sen A.S setiap satu.'

In [8]:
all_special_ids = [0, 1, 2]

In [10]:
en_[0]

'On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.'

In [9]:
input_ids = tokenizer.encode(f'terjemah ke Melayu: {en_[0]}{tokenizer.eos_token}', return_tensors = 'pt')
outputs = model.generate(input_ids, max_length = 1024, num_beams=5, early_stopping=True)
outputs = [i for i in outputs[0] if i not in all_special_ids]
print(tokenizer.decode(outputs, spaces_between_special_tokens = False).strip())

Pada hari Isnin, saintis dari Sekolah Perubatan Universiti Stanford mengumumkan penciptaan alat diagnostik baharu yang boleh menyusun sel mengikut jenis: cip cetakan kecil yang boleh dihasilkan menggunakan pencetak inkjet standard untuk kira-kira satu sen A.S. setiap satu.


In [12]:
for i in tqdm(range(len(ms_))):
    filename = os.path.join('small-en-ms', f'{i}.json')
    
    if os.path.exists(filename):
        continue
    
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }
    
    input_ids = tokenizer.encode(f'terjemah ke Melayu: {en_[i]}{tokenizer.eos_token}', return_tensors = 'pt')
    outputs = model.generate(input_ids, max_length = 1024, num_beams=5, early_stopping=True)
    outputs = [i for i in outputs[0] if i not in all_special_ids]
    r = tokenizer.decode(outputs, spaces_between_special_tokens = False).strip()

    with open(filename, 'w') as fopen:
        json.dump({'text': en_[i], 'r': r}, fopen)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 997/997 [06:46<00:00,  2.45it/s]


In [13]:
from sacrebleu.metrics import BLEU, CHRF, TER

chrf = CHRF(word_order = 2)

In [14]:
predicted = []
for i in range(len(ms_)):
    filename = os.path.join('small-en-ms', f'{i}.json')
    with open(filename) as fopen:
        d = json.load(fopen)
    predicted.append(d['r'])

In [15]:
score = chrf.corpus_score(predicted, [ms_])
score

chrF2++ = 66.23